# Live Code 3

---



- Andik Al Fauzi - FTDS-RMT-018

# Problem Statement Background

---



Austin City adalah salah satu kota dengan tingkat kriminalitas yang cukup tinggi. Para penegak hukum di kota ini sedang berusaha semaksimal mungkin untuk mengurangi tingkat kriminalitas tersebut dengan cara menganalisa data-data kriminalitas yang sudah terjadi sebelumnya agar di kemudian hari dapat bekerja dengan efektif dan semaksimal mungkin. Berikut breakdown untuk problem statementnya : 
1. Specific

  - Apakah Data yang ditampilkan sudah sesuai dengan problem list point yang dijabarkan
2. Measurable
  - Bagaimana pengukurang yang dilakukan terhadap dataset tersebut?
3. Achiveable
  - Apakah tingkat keberhasilan data tersebut sudah cukup tinggi
4. Relevant
  - Apakah data-data tersebut memiliki keterikatan satu-sama lain
5. Time-bond Goals
  - Berapa lama para penegak hukum dalam menangani suatu kasus

# Data Preparation

---



In [1]:
# Importing library

import pandas as pd
import numpy as np

In [10]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
project_id = 'abiding-state-376306'
client = bigquery.Client(project=project_id)

In [2]:
# Uploading dataset

crimeDF = pd.read_csv('/content/P0LC3_Andik_1.csv')

In [3]:
# Create a copy dataset to make sure the original one is untouchable

copyDF = crimeDF.copy()

# Problem List Point

---



1. Tampilkan tabel crime sampai 5 baris pertama
2. Apa saja jenis tindak kejahatan yang terjadi di kota Austin selama tahun 2016?
3. Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?
4. Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?
5. Di bulan apa di tahun 2016 kasus pencurian (theft) sangat banyak terjadi?
6. Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?
7. Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016? (Hint: Hitung selisih hari dari timestamp hingga clearance date)

# Answer of Problem List Point 1

---



1. Tampilkan tabel crime sampai 5 baris pertama

In [4]:
copyDF.head(5)

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,20165002898,"Austin, TX",NaN,2016-03-14 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-19 12:00:00.000000 UTC,NaN,NaN,2016,NaN
1,20165004843,"Austin, TX",NaN,2016-03-31 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-25 12:00:00.000000 UTC,NaN,NaN,2016,NaN
2,2016250988,"Austin, TX",NaN,2016-09-16 12:00:00.000000 UTC,Not cleared,NaN,RAPE,UK,NaN,NaN,NaN,NaN,Rape,2016-01-25 12:00:00.000000 UTC,NaN,NaN,2016,NaN
3,20165003839,"Austin, TX",NaN,2016-02-09 12:00:00.000000 UTC,Not cleared,NaN,AGG RAPE OF A CHILD,UK,NaN,NaN,NaN,NaN,Rape,2016-01-27 12:00:00.000000 UTC,NaN,NaN,2016,NaN
4,2016321746,"Austin, TX",NaN,2016-03-22 12:00:00.000000 UTC,Not cleared,NaN,RAPE OF A CHILD,UK,NaN,NaN,NaN,NaN,Rape,2016-02-01 12:00:00.000000 UTC,NaN,NaN,2016,NaN


# Answer of Problem List Point 2

---



2. Apa saja jenis tindak kejahatan yang terjadi di kota Austin selama tahun 2016?

In [17]:
# Query to get criminal type in 2016

df2016 = client.query('''
SELECT address, primary_type, clearance_date, timestamp,
FROM `bigquery-public-data.austin_crime.crime`
WHERE timestamp between '2016-01-01' AND '2016-12-31'
ORDER BY clearance_date
''').to_dataframe()

In [19]:
# Find unique key of criminal

df2016.primary_type.unique()

array(['Rape', 'Theft', 'Robbery', 'Burglary', 'Agg Assault',
       'Auto Theft', 'Murder'], dtype=object)

- Insight :

---



- Selama tahun 2016 tindah kejahatan yang terjadi adalah `Rape`, `Theft`, `Robbery`, `Burglary`, `Agg Asault`, `Auto-Theft`, `Murder`

# Answer of Problem List Point 3

---



Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?

In [20]:
# Query to get clearance status in 2016

clearanceDF = client.query('''
select address, clearance_status, clearance_date 
FROM `bigquery-public-data.austin_crime.crime`
WHERE clearance_date between '2016-01-01' and '2016-12-31'
AND clearance_status LIKE 'Cleared%%'
ORDER BY clearance_date
''').to_dataframe()

In [22]:
# find a unique key of clearance status
clearanceDF.clearance_status.unique()

array(['Cleared by Arrest', 'Cleared by Exception'], dtype=object)

In [25]:
# function to find the summary of clearence status = clear
arrestDF = (clearanceDF['clearance_status'] =='Cleared by Arrest').sum()
exceptionDF = (clearanceDF['clearance_status'] =='Cleared by Exception').sum()

# print data
print('Case with status Cleared by Arrest : ', arrestDF)
print('Case with status Cleared by Exception :', exceptionDF)

Case with status Cleared by Arrest :  5071
Case with status Cleared by Exception : 992


- Insight : 

---



- Dari data diatas dapat dilihat bahwa penanganan kasus dengan status `Cleared by Arrest` mencapai angka `5,071`, ini manandakan bahwa petugas dengan sigap manangkap para pelaku tindak kejahatan. sementara dengan status `Cleared by Exception` sebanyak `992`, hal ini terjadi bisa saja ada toleransi yang diberikan oleh petugas, semisal pelaku kejahatan masih dibawah umur ataupun ada gangguan mental yang dialami oleh pelaku kejahatan. 

# Answer of Problem List Point 4

---



4. Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?

In [27]:
# Query SQL to get status not cleared from clearance_status

notClearDF = client.query('''
select address, clearance_status, clearance_date, primary_type 
FROM `bigquery-public-data.austin_crime.crime`
WHERE clearance_date between '2016-01-01' and '2016-12-31'
AND clearance_status LIKE 'Not%%'
ORDER BY clearance_date
''').to_dataframe()

In [29]:
# find the criminal category
notClearDF.primary_type.unique()

array(['Theft', 'Auto Theft', 'Agg Assault', 'Burglary', 'Robbery',
       'Rape'], dtype=object)

In [31]:
# function to find summary of each criminal category
theftDF = (notClearDF['primary_type'] == 'Theft').sum()
autoDF = (notClearDF['primary_type'] == ' Auto Theft').sum()
aggDF = (notClearDF['primary_type'] == 'Agg Asault').sum()
burgDF = (notClearDF['primary_type'] == 'Bruglary').sum()
robbDF = (notClearDF['primary_type'] == 'Robbery').sum()
rapeDF = (notClearDF['primary_type'] == 'Rape').sum()

# Print Data
print('Jumlah kasus Theft sebanyak : ', theftDF)
print('Jumlah kasus Auto Theft sebanyak : ', autoDF)
print('Jumlah kasus Agg Asault sebanyak : ', aggDF)
print('Jumlah kasus Burglary sebanyak : ', burgDF)
print('Jumlah kasus Robbery sebanyak : ', robbDF)
print('Jumlah kasus Rape sebanyak : ', rapeDF)

Jumlah kasus Theft sebanyak :  21945
Jumlah kasus Auto Theft sebanyak :  0
Jumlah kasus Agg Asault sebanyak :  0
Jumlah kasus Burglary sebanyak :  0
Jumlah kasus Robbery sebanyak :  559
Jumlah kasus Rape sebanyak :  330


- Insight : 

---



- Kasus yang belum selesai ditangani sebanyak `21,945` adalah kasus dengan jenis `Theft` yang mana menduduki peringkat pertama. Hal ini sulit dilakukan karena kemungkinan para pelaku kejahatan dengan cepat untuk menjual/menyalurkan hasil kejahatannya di pasar gelap, dan mungkin juga para pelaku melarikan diri ke daerah lain sehingga para petugas kesulitan untuk melacak para pelaku. Sedangkan untuk kasus `Auto Theft`, `Agg Asault` dan `Burglary` semua sudah selesai ditangani dengan baik.

# Answer of Problem List Point 5

---



5. Di bulan apa di tahun 2016 kasus pencurian (theft) sangat banyak terjadi?

In [40]:
# Query to get month in case `Theft`
monthTheftDF = client.query('''
SELECT 
  EXTRACT(Month FROM clearance_date) AS Month,
  COUNT(*) AS TotalTheft
FROM `bigquery-public-data.austin_crime.crime`
WHERE primary_type LIKE '%Theft%'
AND EXTRACT(year FROM clearance_date) = 2016
Group BY Month 
ORDER BY Month
''').to_dataframe()

In [43]:
monthTheftDF

,Month,TotalTheft
0,1,2350
1,2,2133
2,3,2243
3,4,2164
4,5,2358
5,6,2211
6,7,1989
7,8,2553
8,9,2128
9,10,2168


- Insight : 

---



- Bulan August 2016 merupakan bulan terbanyak kasus `Theft` dengan total `2,553` kasus, dan bulan terendah di bulan July 2016 dengan total kasus `1,989`

# Answer of Problem List Point 6

---



6. Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?

In [50]:
districDF = client.query('''
SELECT district,
count(*) AS TotalTheft
FROM `bigquery-public-data.austin_crime.crime`
where timestamp between '2016-01-01' And '2016-12-31'
group by district
order by TotalTheft
''').to_dataframe()

In [51]:
districDF

,district,TotalTheft
0,88,26
1,UK,134
2,AP,162
3,G,2640
4,C,2822
5,A,4139
6,I,4244
7,H,4251
8,B,4267
9,F,4495


- Insight : 

---



- Di `District` E paling banyak kasus tindak kejahatan dengan total `4,772` kasus kejahatan

# Answer of Problem List Point 7

---



7. Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016? (Hint: Hitung selisih hari dari timestamp hingga clearance date)

In [59]:
#Query to get average ratio
averageDF = client.query('''
SELECT
  AVG(TIMESTAMP_DIFF(clearance_date, timestamp, DAY)) AS avg_time_to_clear,
  COUNT(*) AS num_cases
FROM
  `bigquery-public-data.austin_crime.crime`
WHERE
  primary_type = 'Theft' AND EXTRACT(YEAR FROM timestamp) = 2016
  AND clearance_date IS NOT NULL
''').to_dataframe

In [57]:
averageDF()

,avg_time_to_clear,num_cases
0,17.628652,24955


- Insight :
- Rata-rata penyelasaian kasus adalah 17,6 bulan dari total kasus di 2016 sebanyak `24,955`

# Problem Statement Solving

---



1. Spesific
- Data-data yang ada sudah disajikan dengan spesifik mungkin di range tahun 2016
2. Measurable
- Data-data tersebut dapat diukur dengan baik, baik dari segi total kejahatan per distrik, per bulan maupun dari jenis kejahatannya itu sendiri
3. Achiveable
- Tingkat keberhasilan petugas dalam menangani kasus kejahatan pun termasuk tinggi
4. Relevant
- Data tersaji sudah sangat relevant, mengingat pembagian datanya sudah baik
5. Time Bond Goals
- Tujuan dari data ini adalah agar para petugas dapat sebaik mungkin untuk meminimalisir tingkat kejahatan dengan range waktu yang tegolong singkat

# Conclusion

---



Para Petugas sudah dengan sangat baik menangani tindak kejahatan yg terjadi dikota austin, dengan bukti rata-rata penyelesaian di angkat 17,6 per 24,995 kasus. Kinerja ini diiringi dengan semangat agar tingkat kejahatan kota Austin di 2017 dapat menurun dibanding dengan tahun 2016